In [1]:
from IPython.display import Markdown
import pandas as pd
from tulip.core.collection import TulipCollection
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.haver import HaverClient as hc
from tulip.plots import plot_line, plot_lines
from tulip.data.gs import GSClient as gs
from tulip.analysis.country_related.analytics import summarize_gs_eco_fct

import pycountry

ISO_2 = "BR"
haver_code = 223
pyctry = pycountry.countries.get(alpha_2=ISO_2)
import plotly.offline as pyo

pyo.init_notebook_mode(connected=True)

Haver path setting remains unchanged.



## Brazil

### Economic Forecasts (Brokers)

In [2]:
gs_eco_fct = gs.get_eco_forecast(geographyId=ISO_2)
gs_summary = summarize_gs_eco_fct(gs_eco_fct)
gs_summary = gs_summary[~gs_summary.index.str.contains("ngdp")].to_frame().T
gs_summary.style.set_caption(f"Goldman {pyctry.name} Economic Forecasts").format(
    precision=2
)

metric,core_cpi,cpi_avg,current_account,rgdp_qoq,rgdp_yoy,output_gap
forecastValue,nan,5.13,nan,1.00,2.59,nan


### Current Activity Indicator (Goldman)

In [3]:
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=ISO_2,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)
cai_series_soft_vs_hard = cai_series_soft_vs_hard.set_index("metricName", append=True)[
    "metricValue"
].unstack("metricName")
cai_series_soft_vs_hard.columns = ["Hard", "Soft", "Headline"]
cai_plot = plot_lines(
    cai_series_soft_vs_hard,
    show_0=True,
    title=f"<b>{pyctry.name} Current Activity Indicator</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    years_limit=4,
)
cai_plot

### Growth Stats

#### Real GDP Growth

In [4]:
real_gdp_yoy = hc.get_series("S223NGPC@EMERGE")
real_gdp_yoy.ts = real_gdp_yoy.ts.pct_change(4)
real_gdp_yoy.plot(show_0=True, tick_format="0.0%")

#### GDP Detail:

In [5]:
growth_stats = {
    "Real GDP": f"yryr%(S{haver_code}NGPC@EMERGE)",  # Real Gross Domestic Product at Market Prices
    "Fixed Investment": f"yryr%(S{haver_code}NFC@EMERGE)",  # GDP: Gross Fixed Capital Formation
    #  "Inventories":          f"S{haver_code}NBC@EMERGE",           # GDP: Investment in Inventories
    # "Consumption":          f"yryr%(S{haver_code}NCC@EMERGE)",    # GDP: Final Consumption Expenditure
    "Private Consumption": f"yryr%(S{haver_code}NCPC@EMERGE)",  # [Brazil: GDP: Private Consumption (SA, Mil.Chained.1995.Reais)]
    "Public Consumption": f"yryr%(S{haver_code}NCGC@EMERGE)",  #  [Brazil: GDP: Government Consumption (SA, Mil.Chained.1995.Reais)]
    "Exports": f"yryr%(S{haver_code}NXC@EMERGE)",  # GDP: Exports of Goods and Services
    "Imports": f"yryr%(S{haver_code}NMC@EMERGE)",  # GDP: Imports of Goods and Services
}

growth_stats_collection = []
for k, v in growth_stats.items():
    growth_stats_collection.append(hc.get_series(v))

growth_stats_collection = TulipCollection(growth_stats_collection)
# todo: be able to change to down only one gtime series

growth_stats_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: Gross Domestic Product (SA, Mil.Chained.1995.Reais)",2.6,2025-06-30,3.7,-1.2,-0.43,-1.00,-0.28,2025-11-12 19:15:00
"Brazil: GDP: Gross Fixed Capital Formation (SA, Mil.Chained.1995.Reais)",4.0,2025-06-30,9.1,-5.1,-0.76,-6.8,-1.6,2025-11-12 19:15:00
"Brazil: GDP: Private Consumption (SA, Mil.Chained.1995.Reais)",1.8,2025-06-30,2.5,-0.76,-0.27,-3.7,-3.2,2025-11-12 19:15:00
"Brazil: GDP: Government Consumption (SA, Mil.Chained.1995.Reais)",0.45,2025-06-30,1.1,-0.63,-0.16,-0.74,-0.82,2025-11-12 19:15:00
"Brazil: GDP: Exports of Goods & Services (SA, Mil.Chained.1995.Reais)",1.9,2025-06-30,2.6,-0.65,-0.08,1.0,-2.4,2025-11-12 19:15:00
"Brazil: GDP: Imports of Goods & Services (SA, Mil.Chained.1995.Reais)",4.0,2025-06-30,16.2,-12.2,-1.1,-11.0,-10.0,2025-11-12 19:15:00


In [6]:
growth_stats_collection.dashboard.plots(show_0=True)

#### Nominal GDP:

In [7]:
#### Nominal GDP:
hc.get_series(f"yryr%(H{haver_code}NGDP@EMERGE)").plot(
    show_0=True, years_limit=4
)  # Nominal GDP Growth

### Trade

In [8]:
trade_stats = {
    "Current Account Balance % GDP": f"H{haver_code}BCPG@EMERGE",
}

trade_collection = []
for k, v in trade_stats.items():
    trade_collection.append(hc.get_series(v))

trade_collection = TulipCollection(trade_collection)
trade_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: BOP: Current Account Balance as a Percentage of GDP (SA, %)",-3.3,2025-06-30,-3.6,0.36,0.32,0.38,-0.25,2025-11-12 19:15:00


In [9]:
trade_collection.dashboard.plots(show_0=True)

### Surveys

Fundacion Geturlio Vargas are the most tracked Sentiment Indicators

In [10]:
fgv_surveys = {
    "Business industry": {
        "confidence_index": "S223VBT@EMERGELA",
        "current_condition": "S223VBTC@EMERGELA",
        "expectations_index": "S223VBTE@EMERGELA",
    },
    "Manufacturing (CNAE 2.0)": {
        "confidence_index": "S223VM@EMERGELA",
        "current_condition": "S223VMC@EMERGELA",
        "expectations_index": "S223VME@EMERGELA",
    },
    "Services": {
        "confidence_index": "S223VS@EMERGELA",
        "current_condition": "S223VSC@EMERGELA",
        "expectations_index": "S223VSE@EMERGELA",
    },
    "Trade/commerce (TCOI)": {
        "confidence_index": "S223VR@EMERGELA",
        "current_condition": "S223VRC@EMERGELA",
        "expectations_index": "S223VRE@EMERGELA",
    },
}


fgv_surveys_collection = []
for category, metrics in fgv_surveys.items():
    for metric_name, code in metrics.items():
        fgv_surveys_collection.append(hc.get_series(code))

fgv_surveys_collection = TulipCollection(fgv_surveys_collection)
fgv_surveys_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: Business Confidence Index (SA, Points)",89.5,2025-10-31,89.6,-0.10,-0.03,-3.2,-7.8,2025-11-12 19:15:00
"Brazil: Business Confidence Index, Current Condition (SA, Points)",92.8,2025-10-31,93.3,-0.50,-0.18,-2.8,-5.2,2025-11-12 19:15:00
"Brazil: Business Confidence Index, Expectations Index (SA, Points)",86.2,2025-10-31,85.9,0.30,0.08,-3.6,-10.4,2025-11-12 19:15:00
"Brazil: Manufacturing Industry Confidence Index (SA, Points)",89.8,2025-10-31,90.5,-0.70,-0.19,-8.2,-10.1,2025-11-12 19:15:00
"Brazil: Manufacturing Industry Confidence Index, Current Condition (SA, Points)",94.2,2025-10-31,95.0,-0.80,-0.22,-5.9,-8.5,2025-11-12 19:15:00
"Brazil: Manufacturing Industry Confidence Index, Expectations Index (SA, Points)",85.4,2025-10-31,86.1,-0.70,-0.16,-10.6,-11.7,2025-11-12 19:15:00
"Brazil: Services Confidence Index (SA, Points)",88.9,2025-10-31,89.0,-0.10,-0.03,-1.5,-6.7,2025-11-12 19:15:00
"Brazil: Services Confidence Index, Current Condition (SA, Points)",92.9,2025-10-31,93.9,-1.0,-0.35,-0.90,-3.7,2025-11-12 19:16:00
"Brazil: Services Confidence Index, Expectations Index (SA, Points)",85.0,2025-10-31,84.2,0.80,0.18,-2.2,-9.9,2025-11-12 19:16:00
"Brazil: Trade/Commerce Confidence Index (SA, Points)",86.2,2025-10-31,84.7,1.5,0.39,-1.3,-3.8,2025-11-12 19:16:00


In [11]:
fig = plot_lines(
    [ts.ts for ts in fgv_surveys_collection[0:3]],
    title="Brazil: Business Manufacturing  Index",
)
fig.update_traces(name="Confidence Index", selector=dict(name="S223VBT@EMERGELA"))
fig.update_traces(name="Current Condition", selector=dict(name="S223VBTC@EMERGELA"))
fig.update_traces(name="Expectations Index", selector=dict(name="S223VBTE@EMERGELA"))
fig.show()

In [12]:
fig = plot_lines(
    [ts.ts for ts in fgv_surveys_collection[4:6]],
    title="Brazil: Business Manufacturing Index",
)
fig.update_traces(name="Confidence Index", selector=dict(name="S223VM@EMERGELA"))
fig.update_traces(name="Current Condition", selector=dict(name="S223VMC@EMERGELA"))
fig.update_traces(name="Expectations Index", selector=dict(name="S223VME@EMERGELA"))
fig.show()

Other Surveys

In [13]:
surveys = {
    "Consumer Con`fidence Index": "H223VCC@EMERGE ",
    "Business Confidence Index": "N223VB@EMERGE",
    "Business Prospects Index": "N223VBE@EMERGE ",
}

surveys_collection = []
for k, v in surveys.items():
    surveys_collection.append(hc.get_series(v))

surveys_collection = TulipCollection(surveys_collection)

surveys_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: Consumer Confidence Index (SA, >100=Optimism)",110.7,2025-09-30,112.7,-2.0,-0.44,-3.8,-12.8,2025-11-12 19:16:00
Brazil: ICEI: Business Confidence Index (50+=Growth),47.2,2025-10-31,46.2,1.0,0.33,-0.80,-6.0,2025-11-12 19:16:00
"Brazil: ICEI: Business Confidence Index, Expectations (50+=Growth)",49.1,2025-10-31,48.3,0.80,0.26,-1.6,-6.3,2025-11-12 19:16:00


In [14]:
surveys_collection.dashboard.plots(years_limit=3)

### Employment

In [15]:
# Employment metrics
employment = {
    "Unemployment Rate": f"H{haver_code}ELUR@EMERGELA ",
    "Workfoce Participation": f"H{haver_code}ELPR@EMERGELA ",
    "Net Job Creation": f"H{haver_code}E5@EMERGELA ",
}

employment_collection = []
for k, v in employment.items():
    employment_collection.append(hc.get_series(v))

employment_collection = TulipCollection(employment_collection)
# todo: fix direction
employment_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: Unemployment Rate, 3-Month Moving Average (EOP, SA, %)",5.7,2025-09-30,5.7,0.00,0.00,-0.80,-0.80,2025-11-12 19:16:00
"Brazil: Workforce Participation, 3-Month Moving Average (EOP, SA, %)",62.1,2025-09-30,62.2,-0.10,-0.35,-0.20,-0.10,2025-11-12 19:16:00
"Brazil: Registered Jobs Created (SA, Units.)","79,096",2025-09-30,"27,128","51,968",0.69,"-90,942","-36,321",2025-11-12 19:16:00


In [16]:
employment_collection.dashboard.plots()

### Output Gap

In [17]:
# Output gap metrics
output_gap = {
    "Manufacturing Capacity Utilization": "S223DUM@EMERGELA",
    "CNI Capacity Utilization": "S223UCC@EMERGELA",
}

output_gap_collection = []
for k, v in output_gap.items():
    output_gap_collection.append(hc.get_series(v))

output_gap_collection = TulipCollection(output_gap_collection)
output_gap_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: Manufacturing Capacity Utilization (SA, %)",81.9,2025-10-31,82.6,-0.70,-0.44,-1.1,-0.60,2025-11-12 19:16:00
"Brazil: CNI: Capacity Utilization (SA, %)",77.9,2025-09-30,78.3,-0.40,-0.71,-0.70,-1.6,2025-11-12 19:16:00


In [18]:
output_gap_collection.dashboard.plots()

### Prices

The Central Bank of Brazil (Banco Central do Brasil) targets the IPCA (Índice Nacional de Preços ao Consumidor Amplo), or Broad National Consumer Price Index, as its official inflation measure. This index is calculated by the Brazilian Institute of Geography and Statistics (IBGE) and serves as the primary benchmark for the country's inflation-targeting regime. The inflation target is set in terms of the year-over-year rate of increase in the IPCA .
The IPCA-15, on the other hand, is a mid-month preview of the IPCA. While it provides timely insights into inflation trends and is closely monitored by policymakers and analysts, it is not the official target for monetary policy. The IPCA-15 differs from the IPCA primarily in its data collection period and geographic coverage, offering an early indication of inflation but not serving as the formal benchmark .
valorinternational


In [19]:
prices = {
    "IPCA-15": "yryr%(N2231P@EMERGELA)",
    "CPI YoY": f"yryr%(S{haver_code}IP@EMERGELA)",
    "Wages YoY": f"yryr%(N{haver_code}EM@EMERGELA)",
    "Commodity Prices": f"yryr%(N{haver_code}PZRB@EMERGELA)",
    "Inflation Expectations 1Y": "N223VCVI@EMERGELA",
    "Inflation Expectations 1Y FOCUS": "I223ICA@EMERGELA",
    "Target Inflation": "A223VUCP@EMERGELA",
}

prices_collection = []
for k, v in prices.items():
    prices_collection.append(hc.get_series(v))

prices_collection = TulipCollection(prices_collection)
prices_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: CPI: General Index [IPCA-15] (NSA, Dec 1993=100)",4.9,2025-10-31,5.3,-0.38,-0.02,-0.55,0.48,2025-11-12 19:16:00
"Brazil: Consumer Price Index: Extended IPCA (SA, 2015=100)",4.7,2025-10-31,5.2,-0.49,-1.1,-0.90,-0.09,2025-11-12 19:16:00
"Brazil: Avg Monthly Earnings of All Jobs: 3-Mo Mv Avg (EOP, R$)",9.4,2025-09-30,8.7,0.66,0.83,0.85,1.2,2025-11-12 19:16:00
"Brazil: Commodity Research Bureau Commodity Index (NSA, Dec-05=100)",9.9,2024-06-30,3.3,6.6,1.1,21.5,24.7,2025-11-12 19:16:00
"Brazil: Consumer Inflation Expectations for Next 12 Months (NSA, %)",6.3,2025-10-31,6.1,0.20,0.60,-0.10,0.20,2025-11-12 19:16:00
"Brazil: FOCUS Survey: IPCA: Inflation Forecast Next 12 Months: Med (AVG,%)",4.2,2025-10-31,4.4,-0.21,-0.57,-0.93,0.08,2025-11-12 19:16:00
Brazil: Inflation Target: Upper Limit (%),4.5,2026-12-31,4.5,0.00,0.00,0.00,0.00,2025-11-12 19:16:00


In [20]:
fig = plot_lines(
    [ts.ts for ts in prices_collection],
    title="",
    years_limit=5,
    tick_suffix="%",
    default_y_range=(-5, 18),
    figsize=(800, 600),
)
fig.update_traces(
    name="CPI Core YoY (IPCA-15)",
    line_width=4,
    selector=dict(name="yryr%(N2231P@EMERGELA)"),
)
fig.update_traces(
    name="CPI YoY (IPCA)",
    line_width=4,
    selector=dict(name=f"yryr%(S{haver_code}IP@EMERGELA)"),
)
fig.update_traces(
    name="Wages YoY", selector=dict(name=f"yryr%(N{haver_code}EM@EMERGELA)")
)
fig.update_traces(
    name="Commodity Prices",
    selector=dict(name=f"yryr%(N{haver_code}PZRB@EMERGELA)"),
    visible="legendonly",
)
fig.update_traces(name="Expectations", selector=dict(name="N223VCVI@EMERGELA"))
fig.update_traces(
    name="Expectations (Focus survey)", selector=dict(name="I223ICA@EMERGELA")
)
fig.update_traces(
    name="Upper Inflation Target", selector=dict(name="A223VUCP@EMERGELA")
)
fig.show()

##### Double click on Focus survey
The Central Bank of Brazil conducts the weekly Focus Survey, aggregating forecasts from over 100 financial institutions. This survey provides median projections for key economic indicators, including the IPCA inflation rate. Within this framework, the "Top 5" ranking highlights institutions with the most accurate short-, medium-, and long-term forecasts, offering a benchmark for forecasting performance

In [21]:
focus_survey = {
    "Key1": "I223ICA@EMERGELA",
    "Key2": "I223TCA@EMERGELA",
    "Key4": "C223TCA@EMERGELA",
    "Key6": "M223TCA@EMERGELA",
    "Key7": "L223TCA@EMERGELA",
}

focus_survey_collection = []
for k, v in focus_survey.items():
    focus_survey_collection.append(hc.get_series(v))

focus_survey_collection = TulipCollection(focus_survey_collection)
focus_survey_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: FOCUS Survey: IPCA: Inflation Forecast Next 12 Months: Med (AVG,%)",4.2,2025-10-31,4.4,-0.21,-0.57,-0.93,0.08,2025-11-12 19:16:00
"Brazil: FOCUS Survey: IPCA: Infl Fcast Next 12 Mths:[Alt]Med(AVG,%)",4.1,2025-10-31,4.4,-0.22,-0.76,-0.86,0.06,2025-11-12 19:16:00
"Brazil: FOCUS Survey [Top 5 Short Term]: IPCA 12 Month Forecast: Median (AVG,%)",4.2,2025-10-31,4.4,-0.28,-0.85,-1.1,0.23,2025-11-12 19:16:00
"Brazil: FOCUS Survey [Top 5 Medium Term]: IPCA 12 Month Forecast: Median(AVG,%)",4.1,2025-10-31,4.2,-0.10,-0.30,-1.3,0.09,2025-11-12 19:16:00
"Brazil: FOCUS Survey [Top 5 Long Term]: IPCA 12 Month Forecast: Median (AVG,%)",4.4,2025-10-31,4.8,-0.46,-1.3,-1.1,0.34,2025-11-12 19:16:00


In [22]:
plot_lines(focus_survey_collection.ts, years_limit=1)
fig = plot_lines(focus_survey_collection.ts, years_limit=1, logo=False, tick_suffix="%")
fig.update_traces(name="IPCA next year", selector=dict(name="I223ICA@EMERGELA"))
fig.update_traces(
    name="IPCA next year, forecasts in the last 5 working days",
    selector=dict(name="I223TCA@EMERGELA"),
)
fig.update_traces(
    name="Top 5 Short term forecasts", selector=dict(name="C223TCA@EMERGELA")
)
fig.update_traces(
    name="Top 5 medium term forcasts", selector=dict(name="M223TCA@EMERGELA")
)
fig.update_traces(
    name="Top 5 long term forecasts", selector=dict(name="L223TCA@EMERGELA")
)
fig.show()


### Breakeven expectations

In [23]:
breakeven_collection = [
    bb.get_series(f"BRGGBE{i} Index") for i in ["01", "02", "05", "10"]
]
breakeven_collection = TulipCollection(breakeven_collection)
breakeven_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
Brazil Breakeven 1 Year,3.8,2025-11-12,3.9,-0.10,-0.47,-1.1,-2.9,-,2025-11-12 19:16:00
Brazil Breakeven 2 Year,5.1,2025-11-12,5.2,-0.06,-0.49,-0.81,-1.2,-,2025-11-12 19:16:00
Brazil Breakeven 5 Year,5.7,2025-11-12,5.6,0.02,0.18,-0.71,-0.59,-,2025-11-12 19:16:00
Brazil Breakeven 10 Year,6.2,2025-11-12,6.2,0.02,0.15,-0.34,0.14,-,2025-11-12 19:16:00


In [24]:
bei = breakeven_collection.ts
bei.columns = ["1Y Breakeven", "2Y Breakeven", "5Y Breakeven", "10Y Breakeven"]
bei["IPCA-15"] = prices_collection[0].ts.resample("B").ffill()
bei["IPCA"] = prices_collection[1].ts.resample("B").ffill()
fig = plot_lines(
    bei,
    title="Brazil Breakeven Inflation vs Realized Inflation",
    years_limit=5,
    tick_suffix="%",
    default_y_range=(-5, 18),
    figsize=(800, 400),
    logo=False,
)
fig.update_traces(
    name="1Y Breakeven", selector=dict(name="1Y Breakeven"), visible="legendonly"
)
fig.update_traces(
    name="2Y Breakeven", selector=dict(name="2Y Breakeven"), visible="legendonly"
)
fig.update_traces(name="5Y Breakeven", selector=dict(name="5Y Breakeven"))
fig.update_traces(
    name="10Y Breakeven", selector=dict(name="10Y Breakeven"), visible="legendonly"
)
fig.update_traces(name="IPCA-15", selector=dict(name="IPCA-15"), line=dict(dash="dot"))
fig.update_traces(name="IPCA", selector=dict(name="IPCA"), line=dict(dash="dot"))
fig.show()


### Credit Creation

In [25]:
ngdp = hc.get_series("H223MGDP@EMERGELA").ts.rolling(12).sum()

In [26]:
### Credit Creation
credit = {
    "Credit to Non-Financial Sector": f"N{haver_code}FCCT@EMERGELA",
    "Loans": f"N{haver_code}FC@EMERGELA",
    "Loans by National Financial System": f"N{haver_code}FCT@EMERGELA",
    "Loans by Other Financial Corporations": f"N{haver_code}FCO@EMERGELA",
    "Loans by Government Funds": f"N{haver_code}FCV@EMERGELA",
    "Debt Securities": f"N{haver_code}FCD@EMERGELA",
    "Government Debt Securities": f"N{haver_code}FCVD@EMERGELA",
    "Corporate Debt Securities": f"N{haver_code}FCFD@EMERGELA",
    "Securitization": f"N{haver_code}FCSD@EMERGELA",
    "External Debt": f"N{haver_code}FCX@EMERGELA",
    "External Loans": f"N{haver_code}FCXL@EMERGELA",
    "External Market Debt Securities": f"N{haver_code}FCXE@EMERGELA",
    "Domestic Market External Debt Securities": f"N{haver_code}FCXD@EMERGELA",
}

credit_collection = []
for k, v in credit.items():
    credit_collection.append(hc.get_series(v))

credit_collection = TulipCollection(credit_collection)
credit_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Brazil: Credit to Nonfinancial Sector (NSA, EOP, Mil.R$)","19,810,556",2025-09-30,"19,767,591","42,965",0.35,"988,565","2,161,826",2025-11-12 19:16:00
"Brazil: Credit to Nonfinancial Sector: Loans (NSA, EOP, Mil.R$)","7,151,966",2025-09-30,"7,079,516","72,450",2.3,"336,644","641,701",2025-11-12 19:16:00
"Brazil: Financial System Credit: Total Balance (NSA,EOP,Mil.R$)","6,842,399",2025-09-30,"6,770,883","71,516",3.0,"323,704","626,221",2025-11-12 19:16:00
"Brazil: Nonfin Sector Credit: Loans Granted by Oth Fin Corp (NSA, EOP, Mil.R$)","216,533",2025-09-30,"212,507","4,026",3.0,"18,214","34,330",2025-11-12 19:16:00
"Brazil: Nonfin Sector Credit: Loans Granted by Govt Funds (NSA, EOP, Mil.R$)","225,028",2025-09-30,"222,399","2,629",2.4,"11,574","18,862",2025-11-12 19:16:00
"Brazil: Credit to Nonfinancial Sector: Debt Securities (NSA, EOP, Mil.R$)","9,414,530",2025-09-30,"9,455,397","-40,867",-0.51,"701,068","1,389,590",2025-11-12 19:16:00
"Brazil: Debt Securities Issued by General Government (NSA, EOP, Mil.R$)","7,058,405",2025-09-30,"7,107,796","-49,391",-0.66,"542,659","1,072,278",2025-11-12 19:16:00
"Brazil: Nonfin Sector Credit: Debt Securities Issued by NFC (NSA, EOP, Mil.R$)","1,327,617",2025-09-30,"1,316,611","11,006",0.97,"94,910","195,109",2025-11-12 19:16:00
"Brazil: Nonfinancial Sector Debt Securitization (NSA, EOP, Mil.R$)","1,028,508",2025-09-30,"1,030,990","-2,482",-0.19,"63,499","122,203",2025-11-12 19:16:00
"Brazil: Credit to Nonfinancial Sector: External Debt (NSA, EOP, Mil.R$)","3,244,060",2025-09-30,"3,232,678","11,382",0.17,"-49,147","130,535",2025-11-12 19:16:00


In [27]:
plot_line(
    credit_collection[0].ts.diff(6).mul(2).div(ngdp, axis=0),
    years_limit=10,
    tick_format="0.0%",
    title="<b>Credit Creation to the Non-financial Sector</b> (%NGDP, 6m chg ann.)",
)

In [28]:
# Loans from banks, Loans from Other Financial Institutions, Loans from Covernment
loans_to_the_private_sector = hc.create_collection(
    ["N223FCT@EMERGELA", "N223FCO@EMERGELA", "N223FCV@EMERGELA", "N223FCFD@EMERGELA"]
)
loans_df = loans_to_the_private_sector.df
loans_df.columns = ["Bank Loans", "Other Loans", "Govt Loans", "Bond Issuance"]
private_credit_growth = loans_df.diff(6).mul(2).div(ngdp, axis=0)
plot_lines(
    private_credit_growth,
    years_limit=10,
    tick_format="0.0%",
    title="<b>Credit Creation to the Private Sector</b> (%NGDP, 6m chg ann.)",
)

### Budget

In [29]:
government_finance = {
    "Public Debt": f"N{haver_code}FDP@EMERGELA",  # Canada: Business Barometer Index: 3 Month Outlook
    "Govt Budget Surplus/Deficit": f"N{haver_code}FGBP@EMERGELA",  # Business Barometer Index: Canada
}

government_finance_collection = []
for k, v in government_finance.items():
    government_finance_collection.append(hc.get_series(v))

government_finance_collection = TulipCollection(government_finance_collection)

government_finance_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Brazil: Gross General Government Debt as % of Annualized GDP (%),76.8,2025-09-30,75.3,1.5,0.59,0.90,1.5,2025-11-12 19:17:00
Brazil: Natl Treasury Performance Summary Surplus[+]/Deficit[-] as % of GDP (%),-8.2,2025-06-30,-3.9,-4.3,-1.3,-1.2,2.6,2025-11-12 19:17:00


In [30]:
government_finance_collection.dashboard.plots(show_0=True)

In [31]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:17_